In [20]:
import numpy as np


# Alternative definition of the DFT, its inverse and adjoint

## Forward transform

In the paper, an alternative definition of the DFT is used. Let $n \in \mathbb{N}$ and $x \in \mathbb{C}^n$. We denote by $n/2$ the quotient of the division of $n$ by $2$ (which corresponds to ``n//2`` in Python). We define:
$$
\mathcal{D}(n) = 
\begin{cases}
\llbracket - n/2, n/2 - 1\rrbracket &\text{ if } n \text{ is even} \\
\llbracket - n/2, n/2 \rrbracket &\text{ else}
\end{cases}
$$

Then, the new DFT of a signal $x : \mathcal{D}(n) \to \mathbb{C}$ is defined as:
$$
\forall k \in \mathcal{D}(n), \, \widehat{x}(k) = \sum_{u \in \mathcal{D}(n)} x(u) \exp\left(-2i \pi \dfrac{k u}{n} \right)
$$

A fast computation of $\widehat{x}$ can be obtained using the FFT alongside with ``fftshift`` and ``ifftshift``.

We implement and test it here.

### In 1D:

Naive version, used to test the other one:

In [21]:
def naive_fourier(x):
    n = len(x)
    q_n, r_n = divmod(n, 2)

    u = np.arange(-q_n, q_n + r_n)
    k = np.arange(-q_n, q_n + r_n)

    ku = np.exp(-2j * np.pi * np.einsum("k,u->ku", k, u) / n)
    res = np.einsum("u,ku->k", x, ku)

    return res


Fast version:

In [22]:
def new_fft(x):
    return np.fft.fftshift(np.fft.fft(np.fft.ifftshift(x)))


Tests:

In [23]:
x = np.random.rand(10)
print(np.max(np.abs(naive_fourier(x) - new_fft(x))))
x = np.random.rand(11)
print(np.max(np.abs(naive_fourier(x) - new_fft(x))))


4.440892098500626e-16
1.5565409952966684e-15


### In 2D:

An image of size $(n, m)$ is represented by a function $I : \mathcal{D}(n) \times \mathcal{D}(m) \to \mathbb{C}$. Its DFT is:
$$
\forall (l,k) \in \mathcal{D}(n) \times \mathcal{D}(m), \, \widehat{I}(l, k) = \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(m)} I(u, v) \exp\left(-2i \pi \frac{l u}{n} \right) \exp\left(-2i \pi \frac{k v}{m} \right)
$$

In [24]:
def naive_fourier_2D(a):
    n, m = a.shape

    q_n, r_n = divmod(n, 2)
    u = np.arange(-q_n, q_n + r_n)
    l = np.arange(-q_n, q_n + r_n)

    q_m, r_m = divmod(m, 2)
    v = np.arange(-q_m, q_m + r_m)
    k = np.arange(-q_m, q_m + r_m)

    lu = np.exp(-2j * np.pi * np.einsum("l,u->lu", l, u) / n)
    kv = np.exp(-2j * np.pi * np.einsum("k,v->kv", k, v) / m)

    res = np.einsum("uv,ku,lv->kl", a, lu, kv)

    return res


In [25]:
def new_fft2(x):
    return np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(x)))


In [26]:
shapes = [(4, 4), (4, 5), (5, 4), (5, 5)]

for shape in shapes:
    a = np.random.random(shape)
    print(np.max(np.abs(naive_fourier_2D(a) - new_fft2(a))))


7.101134040375713e-16
1.7763568394002505e-15
1.7763568394002505e-15
2.144196810752504e-15


## Inverse DFT

The definition of the inverse DFT is:
$$
\forall k \in \mathcal{D}(n), \, \mathcal{F}^{-1}(y)(k) = \sum_{u \in \mathcal{D}(n)} y(u) \exp\left(2i \pi \dfrac{k u}{n} \right)
$$

### In 1D:

In [27]:
def new_ifft(x):
    return np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(x)))


In [28]:
x = np.random.rand(10)
print(np.max(np.abs(new_ifft(new_fft(x)) - x)))
x = np.random.rand(11)
print(np.max(np.abs(new_ifft(new_fft(x)) - x)))


2.2247776333697144e-16
1.1102230246251565e-16


In [29]:
x = np.random.rand(10)
print(np.max(np.abs(new_fft(new_ifft(x)) - x)))
x = np.random.rand(11)
print(np.max(np.abs(new_fft(new_ifft(x)) - x)))


1.1633786725261115e-16
2.220446049250313e-16


### In 2D:

In [30]:
def new_ifft2(a: np.ndarray) -> np.ndarray:
    return np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(a)))


In [31]:
x = np.random.rand(10, 10)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))
x = np.random.rand(10, 11)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))
x = np.random.rand(11, 10)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))
x = np.random.rand(11, 11)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))


4.441200131448313e-16
3.3548712112276544e-16
3.332472040936454e-16
3.3306690738754696e-16


## Adjoints

In [32]:
def matrix_dot(a, b):
    return np.sum(np.conjugate(a) * b)


In [33]:
def adj_new_fft(x):
    return new_ifft(x) * np.size(x)


def adj_new_ifft(x):
    return new_fft(x) / np.size(x)


def adj_new_fft2(a):
    return new_ifft2(a) * np.size(a)


def adj_new_ifft2(a):
    return new_fft2(a) / np.size(a)


In [34]:
x = np.random.rand(10)
y = np.random.rand(10)
print(np.abs(np.vdot(new_fft(x), y) - np.vdot(x, adj_new_fft(y))))
x = np.random.rand(11)
y = np.random.rand(11)
print(np.abs(np.vdot(new_fft(x), y) - np.vdot(x, adj_new_fft(y))))


1.1102230246251565e-16
8.326672684688674e-16


In [35]:
x = np.random.rand(10)
y = np.random.rand(10)
print(np.abs(np.vdot(new_ifft(x), y) - np.vdot(x, adj_new_ifft(y))))
x = np.random.rand(11)
y = np.random.rand(11)
print(np.abs(np.vdot(new_ifft(x), y) - np.vdot(x, adj_new_ifft(y))))


1.3877787807814457e-17
1.3877787807814457e-17


In [36]:
shapes = [(4, 4), (4, 5), (5, 4), (5, 5)]
err_list = []

for shape in shapes:
    x = np.random.random(shape)
    y = np.random.random(shape)
    print(np.abs(matrix_dot(new_fft2(x), y) - matrix_dot(x, adj_new_fft2(y))))


8.881784197001252e-16
1.7763568394002505e-15
1.793620188084545e-15
1.698314660101103e-15


In [37]:
shapes = [(4, 4), (4, 5), (5, 4), (5, 5)]
err_list = []

for shape in shapes:
    x = np.random.random(shape)
    y = np.random.random(shape)
    print(np.abs(matrix_dot(new_ifft2(x), y) - matrix_dot(x, adj_new_ifft2(y))))


5.594315114139762e-17
6.938893903907228e-18
6.206335383118183e-17
1.1123893155135927e-16
